In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(r"/Users/sk/Library/CloudStorage/GoogleDrive-sachinkumar.srinivasamurthy@sjsu.edu/Shared drives/DATA 228 Big Data/Project/data/AWS/glueInput")
sorted(os.listdir())

['caFimaNfipInsuranceRecordLevel2009to2022.csv',
 'caFloodEventsDayLevel.csv',
 'caNoaaStationDayLevelWeather2005to2023.csv',
 'caNoaaStationDayLevelWeather2005to2023_V2.csv']

# Weather

In [2]:
dfw= pd.read_csv('caNoaaStationDayLevelWeather2005to2023.csv')
dfw

,_c0,_c1,PRCP,SNOW,SNWD,TMAX,TMIN,lat,lng,state,county_name,fips_code
0,US1CAAL0001,20081007,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
1,US1CAAL0001,20081008,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
2,US1CAAL0001,20081009,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
3,US1CAAL0001,20081010,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
4,US1CAAL0001,20081011,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6591622,USW00094299,20230422,0.0,NaN,NaN,200.0,-27.0,41.4836,-120.5614,CA,Modoc County,49.0
6591623,USW00094299,20230423,0.0,NaN,NaN,194.0,50.0,41.4836,-120.5614,CA,Modoc County,49.0
6591624,USW00094299,20230424,0.0,NaN,NaN,167.0,-16.0,41.4836,-120.5614,CA,Modoc County,49.0
6591625,USW00094299,20230425,0.0,NaN,NaN,200.0,-38.0,41.4836,-120.5614,CA,Modoc County,49.0


In [3]:
dfw.state.value_counts()

CA    6591627
Name: state, dtype: int64

In [4]:
len(dfw['fips_code'].unique())

58

In [5]:
len(dfw['county_name'].unique())

62

## Rename, datetype, drop columns, drop records

In [6]:
dfw['_c1']= pd.to_datetime(dfw['_c1'], format='%Y%m%d')
dfw['fips_code']= dfw['fips_code'].astype('float64')

dfw.rename(columns={"_c0":'stationID', '_c1':'date'}, inplace=True)

dfw.drop(columns=['state'], inplace=True)

## IGNORE THIS

In [7]:
dfw[dfw['county_name'].isna()].shape

(88795, 11)

In [8]:
dfw[dfw['fips_code'].isna()].shape

(448078, 11)

In [9]:
a= dfw[dfw['fips_code'].isna()]
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448078 entries, 388405 to 6472811
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   stationID    448078 non-null  object        
 1   date         448078 non-null  datetime64[ns]
 2   PRCP         321533 non-null  float64       
 3   SNOW         210904 non-null  float64       
 4   SNWD         98309 non-null   float64       
 5   TMAX         288143 non-null  float64       
 6   TMIN         288131 non-null  float64       
 7   lat          448078 non-null  float64       
 8   lng          448078 non-null  float64       
 9   county_name  359283 non-null  object        
 10  fips_code    0 non-null       float64       
dtypes: datetime64[ns](1), float64(8), object(2)
memory usage: 41.0+ MB


In [10]:
a= a[['stationID', 'lat', 'lng']]
a.drop_duplicates(inplace=True)
a

,stationID,lat,lng
388405,US1CAIN0001,37.3626,-118.3883
393758,US1CAIN0004,37.3878,-118.4141
399850,US1CAIN0007,37.3854,-118.4150
400213,US1CAIN0008,36.8061,-118.2014
402015,US1CAIN0009,37.1688,-118.2878
...,...,...,...
6167151,USW00023179,34.7678,-114.6183
6300805,USW00023272,37.7706,-122.4269
6407770,USW00053139,36.6019,-117.1450
6437502,USW00093104,35.6864,-117.6908


In [11]:
from geopy.geocoders import Nominatim

# Create a geolocator object
geolocator = Nominatim(user_agent="my-app")

# Iterate through each row of the dataframe
for index, row in a.iterrows():
    # Get the latitude and longitude values
    lat = row['lat']
    lng = row['lng']
    
    # Use geolocator to get the county name
    location = geolocator.reverse(f"{lat}, {lng}")
    county_name = location.raw['address'].get('county')
    city_name = location.raw['address'].get('city')
    
    # Update the county_name column with the county name
    a.loc[index, 'county_name'] = county_name
    a.loc[index, 'city_name'] = city_name

a

,stationID,lat,lng,county_name,city_name
388405,US1CAIN0001,37.3626,-118.3883,CAL Fire San Bernardino Unit,NaN
393758,US1CAIN0004,37.3878,-118.4141,CAL Fire San Bernardino Unit,NaN
399850,US1CAIN0007,37.3854,-118.4150,CAL Fire San Bernardino Unit,NaN
400213,US1CAIN0008,36.8061,-118.2014,CAL Fire San Bernardino Unit,NaN
402015,US1CAIN0009,37.1688,-118.2878,CAL Fire San Bernardino Unit,NaN
...,...,...,...,...,...
6167151,USW00023179,34.7678,-114.6183,None,None
6300805,USW00023272,37.7706,-122.4269,None,San Francisco
6407770,USW00053139,36.6019,-117.1450,None,None
6437502,USW00093104,35.6864,-117.6908,CAL Fire Kern County,None


In [12]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 388405 to 6472678
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   stationID    135 non-null    object 
 1   lat          135 non-null    float64
 2   lng          135 non-null    float64
 3   county_name  99 non-null     object 
 4   city_name    42 non-null     object 
dtypes: float64(2), object(3)
memory usage: 10.4+ KB


In [13]:
b= a[a['county_name'].isna()]
b= b[b['city_name']=='San Francisco']
b

,stationID,lat,lng,county_name,city_name
1472464,US1CASF0003,37.7834,-122.4701,None,San Francisco
1473488,US1CASF0004,37.7650,-122.4348,None,San Francisco
1478317,US1CASF0008,37.7776,-122.4741,None,San Francisco
1479776,US1CASF0014,37.7598,-122.5065,None,San Francisco
1479929,US1CASF0017,37.7590,-122.3975,None,San Francisco
1480905,US1CASF0020,37.7781,-122.4763,None,San Francisco
1481519,US1CASF0021,37.7272,-122.4096,None,San Francisco
3581966,USC00047767,37.7281,-122.5053,None,San Francisco
6300805,USW00023272,37.7706,-122.4269,None,San Francisco


In [14]:
a= a[a['county_name'].notna()]
a

,stationID,lat,lng,county_name,city_name
388405,US1CAIN0001,37.3626,-118.3883,CAL Fire San Bernardino Unit,NaN
393758,US1CAIN0004,37.3878,-118.4141,CAL Fire San Bernardino Unit,NaN
399850,US1CAIN0007,37.3854,-118.4150,CAL Fire San Bernardino Unit,NaN
400213,US1CAIN0008,36.8061,-118.2014,CAL Fire San Bernardino Unit,NaN
402015,US1CAIN0009,37.1688,-118.2878,CAL Fire San Bernardino Unit,NaN
...,...,...,...,...,...
5770563,USR0000CYUC,34.1233,-116.4078,CAL Fire San Bernardino Unit,None
6035416,USW00003154,33.3042,-117.3550,CAL Fire San Diego Unit,None
6083360,USW00003179,33.9756,-117.6250,CAL Fire San Bernardino Unit,None
6437502,USW00093104,35.6864,-117.6908,CAL Fire Kern County,None


In [15]:
a['county_name'].value_counts()

CAL Fire San Diego Unit              59
CAL Fire San Bernardino Unit         26
CAL Fire Kern County                  6
CAL Fire Lassen Modoc Plumas Unit     4
San Bernardino County                 2
Lassen County                         1
San Diego County                      1
Name: county_name, dtype: int64

### a is useless

In [16]:
b

,stationID,lat,lng,county_name,city_name
1472464,US1CASF0003,37.7834,-122.4701,None,San Francisco
1473488,US1CASF0004,37.7650,-122.4348,None,San Francisco
1478317,US1CASF0008,37.7776,-122.4741,None,San Francisco
1479776,US1CASF0014,37.7598,-122.5065,None,San Francisco
1479929,US1CASF0017,37.7590,-122.3975,None,San Francisco
1480905,US1CASF0020,37.7781,-122.4763,None,San Francisco
1481519,US1CASF0021,37.7272,-122.4096,None,San Francisco
3581966,USC00047767,37.7281,-122.5053,None,San Francisco
6300805,USW00023272,37.7706,-122.4269,None,San Francisco


In [17]:
b['county_name']='San Francisco County'
b['fips_code']=75.0
b

,stationID,lat,lng,county_name,city_name,fips_code
1472464,US1CASF0003,37.7834,-122.4701,San Francisco County,San Francisco,75.0
1473488,US1CASF0004,37.7650,-122.4348,San Francisco County,San Francisco,75.0
1478317,US1CASF0008,37.7776,-122.4741,San Francisco County,San Francisco,75.0
1479776,US1CASF0014,37.7598,-122.5065,San Francisco County,San Francisco,75.0
1479929,US1CASF0017,37.7590,-122.3975,San Francisco County,San Francisco,75.0
1480905,US1CASF0020,37.7781,-122.4763,San Francisco County,San Francisco,75.0
1481519,US1CASF0021,37.7272,-122.4096,San Francisco County,San Francisco,75.0
3581966,USC00047767,37.7281,-122.5053,San Francisco County,San Francisco,75.0
6300805,USW00023272,37.7706,-122.4269,San Francisco County,San Francisco,75.0


## Continue from this

## impute county and fips code for the following stations

In [18]:
dfw.isna().sum()

stationID            0
date                 0
PRCP           1831646
SNOW           3801349
SNWD           5047928
TMAX           2582044
TMIN           2585687
lat                  0
lng                  0
county_name      88795
fips_code       448078
dtype: int64

In [19]:
b['stationID'].unique()

array(['US1CASF0003', 'US1CASF0004', 'US1CASF0008', 'US1CASF0014',
       'US1CASF0017', 'US1CASF0020', 'US1CASF0021', 'USC00047767',
       'USW00023272'], dtype=object)

In [20]:
for i in b['stationID'].unique().tolist():
    dfw.loc[dfw['stationID']==i, 'county_name'] = 'San Francisco County'
    dfw.loc[dfw['stationID']==i, 'fips_code'] = 75.0
dfw.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6591627 entries, 0 to 6591626
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   stationID    6591627 non-null  object        
 1   date         6591627 non-null  datetime64[ns]
 2   PRCP         4759981 non-null  float64       
 3   SNOW         2790278 non-null  float64       
 4   SNWD         1543699 non-null  float64       
 5   TMAX         4009583 non-null  float64       
 6   TMIN         4005940 non-null  float64       
 7   lat          6591627 non-null  float64       
 8   lng          6591627 non-null  float64       
 9   county_name  6525145 non-null  object        
 10  fips_code    6165862 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(2)
memory usage: 553.2+ MB


In [21]:
dfw.isna().sum()

stationID            0
date                 0
PRCP           1831646
SNOW           3801349
SNWD           5047928
TMAX           2582044
TMIN           2585687
lat                  0
lng                  0
county_name      66482
fips_code       425765
dtype: int64

## PAY ATTENTION!!! drop records by subset column only

In [22]:
dfw.isna().sum()

stationID            0
date                 0
PRCP           1831646
SNOW           3801349
SNWD           5047928
TMAX           2582044
TMIN           2585687
lat                  0
lng                  0
county_name      66482
fips_code       425765
dtype: int64

In [23]:
dfw.dropna(subset=['county_name'], inplace=True)
dfw[dfw['fips_code'].isna()]

,stationID,date,PRCP,SNOW,SNWD,TMAX,TMIN,lat,lng,county_name,fips_code
388405,US1CAIN0001,2009-11-09,0.0,0.0,NaN,NaN,NaN,37.3626,-118.3883,CAL Fire San Bernardino Unit,NaN
388406,US1CAIN0001,2009-11-10,0.0,0.0,NaN,NaN,NaN,37.3626,-118.3883,CAL Fire San Bernardino Unit,NaN
388407,US1CAIN0001,2009-11-11,0.0,0.0,NaN,NaN,NaN,37.3626,-118.3883,CAL Fire San Bernardino Unit,NaN
388408,US1CAIN0001,2009-11-12,0.0,0.0,NaN,NaN,NaN,37.3626,-118.3883,CAL Fire San Bernardino Unit,NaN
388409,US1CAIN0001,2009-11-13,0.0,0.0,NaN,NaN,NaN,37.3626,-118.3883,CAL Fire San Bernardino Unit,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6472807,USW00093121,2005-11-26,0.0,0.0,0.0,194.0,94.0,34.3000,-116.1667,CAL Fire San Bernardino Unit,NaN
6472808,USW00093121,2005-11-27,0.0,0.0,0.0,133.0,39.0,34.3000,-116.1667,CAL Fire San Bernardino Unit,NaN
6472809,USW00093121,2005-11-28,0.0,0.0,0.0,156.0,0.0,34.3000,-116.1667,CAL Fire San Bernardino Unit,NaN
6472810,USW00093121,2005-11-29,0.0,0.0,0.0,178.0,28.0,34.3000,-116.1667,CAL Fire San Bernardino Unit,NaN


In [24]:
dfw[dfw['fips_code'].isna()]['county_name'].value_counts()

CAL Fire San Diego Unit              208363
CAL Fire San Bernardino Unit          89697
CAL Fire Lassen Modoc Plumas Unit     48503
CAL Fire Kern County                  12720
Name: county_name, dtype: int64

### cal fire is not needed. it doesnt record rain

In [25]:
dfw.dropna(subset=['fips_code'], inplace=True)
dfw.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165862 entries, 0 to 6591626
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   stationID    6165862 non-null  object        
 1   date         6165862 non-null  datetime64[ns]
 2   PRCP         4460278 non-null  float64       
 3   SNOW         2589729 non-null  float64       
 4   SNWD         1449425 non-null  float64       
 5   TMAX         3733283 non-null  float64       
 6   TMIN         3729616 non-null  float64       
 7   lat          6165862 non-null  float64       
 8   lng          6165862 non-null  float64       
 9   county_name  6165862 non-null  object        
 10  fips_code    6165862 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(2)
memory usage: 564.5+ MB


In [26]:
dfw

,stationID,date,PRCP,SNOW,SNWD,TMAX,TMIN,lat,lng,county_name,fips_code
0,US1CAAL0001,2008-10-07,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1.0
1,US1CAAL0001,2008-10-08,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1.0
2,US1CAAL0001,2008-10-09,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1.0
3,US1CAAL0001,2008-10-10,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1.0
4,US1CAAL0001,2008-10-11,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1.0
...,...,...,...,...,...,...,...,...,...,...,...
6591622,USW00094299,2023-04-22,0.0,NaN,NaN,200.0,-27.0,41.4836,-120.5614,Modoc County,49.0
6591623,USW00094299,2023-04-23,0.0,NaN,NaN,194.0,50.0,41.4836,-120.5614,Modoc County,49.0
6591624,USW00094299,2023-04-24,0.0,NaN,NaN,167.0,-16.0,41.4836,-120.5614,Modoc County,49.0
6591625,USW00094299,2023-04-25,0.0,NaN,NaN,200.0,-38.0,41.4836,-120.5614,Modoc County,49.0


### change dtype to int

In [27]:
dfw['fips_code']= dfw['fips_code'].astype('int')
len(dfw['fips_code'].unique())

58

## agg at day-county-level

In [28]:
dfw['TEMP']= (dfw['TMAX']+dfw['TMIN'])/2
dfw

,stationID,date,PRCP,SNOW,SNWD,TMAX,TMIN,lat,lng,county_name,fips_code,TEMP
0,US1CAAL0001,2008-10-07,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN
1,US1CAAL0001,2008-10-08,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN
2,US1CAAL0001,2008-10-09,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN
3,US1CAAL0001,2008-10-10,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN
4,US1CAAL0001,2008-10-11,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6591622,USW00094299,2023-04-22,0.0,NaN,NaN,200.0,-27.0,41.4836,-120.5614,Modoc County,49,86.5
6591623,USW00094299,2023-04-23,0.0,NaN,NaN,194.0,50.0,41.4836,-120.5614,Modoc County,49,122.0
6591624,USW00094299,2023-04-24,0.0,NaN,NaN,167.0,-16.0,41.4836,-120.5614,Modoc County,49,75.5
6591625,USW00094299,2023-04-25,0.0,NaN,NaN,200.0,-38.0,41.4836,-120.5614,Modoc County,49,81.0


In [29]:
dfw_summary = dfw.groupby(['fips_code','date'])['PRCP', 'TEMP', 'SNOW', 'SNWD'].agg(['min', 'max', 'mean']).reset_index()
dfw_summary

/var/folders/q4/8qvknrhs5d9dq4q2zxqgqzxr0000gn/T/ipykernel_9246/332700576.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfw_summary = dfw.groupby(['fips_code','date'])['PRCP', 'TEMP', 'SNOW', 'SNWD'].agg(['min', 'max', 'mean']).reset_index()


fips_code       date  PRCP                 TEMP                     \
                              min    max   mean    min    max        mean   
0              1 2005-01-01  23.0  109.0   58.9   22.5  105.5   82.571429   
1              1 2005-01-02  43.0  264.0  136.8   17.0  105.5   69.857143   
2              1 2005-01-03   0.0  117.0   38.7   22.0   89.0   69.392857   
3              1 2005-01-04   0.0   13.0    2.3   19.5   88.5   73.357143   
4              1 2005-01-05   0.0   13.0    3.5 -122.5   92.0   55.153846   
...          ...        ...   ...    ...    ...    ...    ...         ...   
387734       115 2023-04-23   0.0    0.0    0.0  156.0  200.0  178.000000   
387735       115 2023-04-24   0.0    0.0    0.0  158.5  186.0  172.250000   
387736       115 2023-04-25   0.0    0.0    0.0  178.0  197.0  187.500000   
387737       115 2023-04-26   0.0    0.0    0.0  197.0  197.5  197.250000   
387738       115 2023-04-27   NaN    NaN    NaN  211.0  211.0  211.000000   

       SNOW           SNWD            
        min  max mean  min  max mean  
0       0.0  0.0  0.0  0.0  0.0  0.0  
1       0.0  0.0  0.0  0.0  0.0  0.0  
2       0.0  0.0  0.0  0.0  0.0  0.0  
3       0.0  0.0  0.0  0.0  0.0  0.0  
4       0.0  0.0  0.0  0.0  0.0  0.0  
...     ...  ...  ...  ...  ...  ...  
387734  0.0  0.0  0.0  0.0  0.0  0.0  
387735  0.0  0.0  0.0  0.0  0.0  0.0  
387736  0.0  0.0  0.0  0.0  0.0  0.0  
387737  0.0  0.0  0.0  NaN  NaN  NaN  
387738  NaN  NaN  NaN  NaN  NaN  NaN  

[387739 rows x 14 columns]

In [30]:
joined_cols = ['_'.join(col).strip() for col in dfw_summary.columns.values]
dfw_summary.columns = joined_cols
dfw_summary

,fips_code_,date_,PRCP_min,PRCP_max,PRCP_mean,TEMP_min,TEMP_max,TEMP_mean,SNOW_min,SNOW_max,SNOW_mean,SNWD_min,SNWD_max,SNWD_mean
0,1,2005-01-01,23.0,109.0,58.9,22.5,105.5,82.571429,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2005-01-02,43.0,264.0,136.8,17.0,105.5,69.857143,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2005-01-03,0.0,117.0,38.7,22.0,89.0,69.392857,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2005-01-04,0.0,13.0,2.3,19.5,88.5,73.357143,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2005-01-05,0.0,13.0,3.5,-122.5,92.0,55.153846,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387734,115,2023-04-23,0.0,0.0,0.0,156.0,200.0,178.000000,0.0,0.0,0.0,0.0,0.0,0.0
387735,115,2023-04-24,0.0,0.0,0.0,158.5,186.0,172.250000,0.0,0.0,0.0,0.0,0.0,0.0
387736,115,2023-04-25,0.0,0.0,0.0,178.0,197.0,187.500000,0.0,0.0,0.0,0.0,0.0,0.0
387737,115,2023-04-26,0.0,0.0,0.0,197.0,197.5,197.250000,0.0,0.0,0.0,NaN,NaN,NaN


In [31]:
dfw_summary.columns=['fips_code', 'date', 'PRCP_min', 'PRCP_max', 'PRCP_mean', 'TEMP_min',
       'TEMP_max', 'TEMP_mean', 'SNOW_min', 'SNOW_max', 'SNOW_mean',
       'SNWD_min', 'SNWD_max', 'SNWD_mean']

dfw_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387739 entries, 0 to 387738
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   fips_code  387739 non-null  int64         
 1   date       387739 non-null  datetime64[ns]
 2   PRCP_min   386578 non-null  float64       
 3   PRCP_max   386578 non-null  float64       
 4   PRCP_mean  386578 non-null  float64       
 5   TEMP_min   380279 non-null  float64       
 6   TEMP_max   380279 non-null  float64       
 7   TEMP_mean  380279 non-null  float64       
 8   SNOW_min   367995 non-null  float64       
 9   SNOW_max   367995 non-null  float64       
 10  SNOW_mean  367995 non-null  float64       
 11  SNWD_min   329356 non-null  float64       
 12  SNWD_max   329356 non-null  float64       
 13  SNWD_mean  329356 non-null  float64       
dtypes: datetime64[ns](1), float64(12), int64(1)
memory usage: 41.4 MB


In [32]:
387739-386578

1161

In [33]:
dfw_summary

,fips_code,date,PRCP_min,PRCP_max,PRCP_mean,TEMP_min,TEMP_max,TEMP_mean,SNOW_min,SNOW_max,SNOW_mean,SNWD_min,SNWD_max,SNWD_mean
0,1,2005-01-01,23.0,109.0,58.9,22.5,105.5,82.571429,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2005-01-02,43.0,264.0,136.8,17.0,105.5,69.857143,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2005-01-03,0.0,117.0,38.7,22.0,89.0,69.392857,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2005-01-04,0.0,13.0,2.3,19.5,88.5,73.357143,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2005-01-05,0.0,13.0,3.5,-122.5,92.0,55.153846,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387734,115,2023-04-23,0.0,0.0,0.0,156.0,200.0,178.000000,0.0,0.0,0.0,0.0,0.0,0.0
387735,115,2023-04-24,0.0,0.0,0.0,158.5,186.0,172.250000,0.0,0.0,0.0,0.0,0.0,0.0
387736,115,2023-04-25,0.0,0.0,0.0,178.0,197.0,187.500000,0.0,0.0,0.0,0.0,0.0,0.0
387737,115,2023-04-26,0.0,0.0,0.0,197.0,197.5,197.250000,0.0,0.0,0.0,NaN,NaN,NaN


In [34]:
dfw_summary.sort_values(by=['fips_code', 'date'], inplace=True)
dfw_summary

,fips_code,date,PRCP_min,PRCP_max,PRCP_mean,TEMP_min,TEMP_max,TEMP_mean,SNOW_min,SNOW_max,SNOW_mean,SNWD_min,SNWD_max,SNWD_mean
0,1,2005-01-01,23.0,109.0,58.9,22.5,105.5,82.571429,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2005-01-02,43.0,264.0,136.8,17.0,105.5,69.857143,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2005-01-03,0.0,117.0,38.7,22.0,89.0,69.392857,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2005-01-04,0.0,13.0,2.3,19.5,88.5,73.357143,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2005-01-05,0.0,13.0,3.5,-122.5,92.0,55.153846,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387734,115,2023-04-23,0.0,0.0,0.0,156.0,200.0,178.000000,0.0,0.0,0.0,0.0,0.0,0.0
387735,115,2023-04-24,0.0,0.0,0.0,158.5,186.0,172.250000,0.0,0.0,0.0,0.0,0.0,0.0
387736,115,2023-04-25,0.0,0.0,0.0,178.0,197.0,187.500000,0.0,0.0,0.0,0.0,0.0,0.0
387737,115,2023-04-26,0.0,0.0,0.0,197.0,197.5,197.250000,0.0,0.0,0.0,NaN,NaN,NaN


In [35]:
dfw_summary.describe()

,fips_code,PRCP_min,PRCP_max,PRCP_mean,TEMP_min,TEMP_max,TEMP_mean,SNOW_min,SNOW_max,SNOW_mean,SNWD_min,SNWD_max,SNWD_mean
count,387739.000000,386578.000000,386578.000000,386578.000000,380279.000000,380279.000000,380279.000000,367995.000000,367995.000000,367995.000000,329356.000000,329356.000000,329356.000000
mean,57.970545,4.466333,45.435483,18.096384,104.190546,180.556348,145.065224,0.357828,4.068164,1.471544,8.815507,148.307588,56.472103
std,33.473488,27.628642,138.060784,63.298884,85.759425,77.172513,74.202116,8.544730,32.108277,14.059584,84.414239,556.305744,237.524121
min,1.000000,0.000000,0.000000,0.000000,-617.000000,-125.000000,-169.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29.000000,0.000000,0.000000,0.000000,50.500000,125.000000,92.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,57.000000,0.000000,0.000000,0.000000,108.500000,175.000000,143.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,87.000000,0.000000,13.000000,1.900000,161.000000,236.500000,199.196154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,115.000000,1372.000000,6248.000000,1659.111111,394.500000,750.000000,402.250000,864.000000,1524.000000,902.000000,3429.000000,11862.000000,4369.000000


In [36]:
dfw_summary.fillna(method='bfill').describe()

,fips_code,PRCP_min,PRCP_max,PRCP_mean,TEMP_min,TEMP_max,TEMP_mean,SNOW_min,SNOW_max,SNOW_mean,SNWD_min,SNWD_max,SNWD_mean
count,387739.000000,387738.000000,387738.000000,387738.000000,387739.000000,387739.000000,387739.000000,387738.000000,387738.000000,387738.000000,387737.000000,387737.000000,387737.000000
mean,57.970545,4.530209,45.420676,18.135250,102.265818,178.572438,143.036359,0.376362,3.913044,1.439169,10.446511,138.659684,55.228703
std,33.473488,27.766038,137.929737,63.288532,86.377920,77.891936,75.157871,8.926184,31.499581,14.087369,87.159996,532.208035,233.271617
min,1.000000,0.000000,0.000000,0.000000,-617.000000,-125.000000,-169.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29.000000,0.000000,0.000000,0.000000,47.500000,122.000000,89.833333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,57.000000,0.000000,0.000000,0.000000,105.500000,175.000000,141.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,87.000000,0.000000,13.000000,1.941176,159.000000,236.000000,198.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,115.000000,1372.000000,6248.000000,1659.111111,394.500000,750.000000,402.250000,864.000000,1524.000000,902.000000,3429.000000,11862.000000,4369.000000


In [37]:
dfw_summary.fillna(method='ffill').describe()

,fips_code,PRCP_min,PRCP_max,PRCP_mean,TEMP_min,TEMP_max,TEMP_mean,SNOW_min,SNOW_max,SNOW_mean,SNWD_min,SNWD_max,SNWD_mean
count,387739.000000,387739.000000,387739.000000,387739.000000,387739.000000,387739.000000,387739.000000,387739.000000,387739.000000,387739.000000,387739.000000,387739.000000,387739.000000
mean,57.970545,4.509309,45.356655,18.098826,106.136160,181.074194,146.248226,0.455017,3.987811,1.517295,8.910832,127.743523,49.560895
std,33.473488,27.665876,137.874975,63.233520,86.222596,76.665419,74.134604,9.223634,31.563543,14.274876,81.749023,516.126832,221.250216
min,1.000000,0.000000,0.000000,0.000000,-617.000000,-125.000000,-169.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29.000000,0.000000,0.000000,0.000000,52.500000,125.000000,93.714286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,57.000000,0.000000,0.000000,0.000000,111.000000,178.000000,144.857143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,87.000000,0.000000,13.000000,2.000000,164.000000,236.000000,202.416667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,115.000000,1372.000000,6248.000000,1659.111111,394.500000,750.000000,402.250000,864.000000,1524.000000,902.000000,3429.000000,11862.000000,4369.000000


In [39]:
dfw_summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387739 entries, 0 to 387738
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   fips_code  387739 non-null  int64         
 1   date       387739 non-null  datetime64[ns]
 2   PRCP_min   386578 non-null  float64       
 3   PRCP_max   386578 non-null  float64       
 4   PRCP_mean  386578 non-null  float64       
 5   TEMP_min   380279 non-null  float64       
 6   TEMP_max   380279 non-null  float64       
 7   TEMP_mean  380279 non-null  float64       
 8   SNOW_min   367995 non-null  float64       
 9   SNOW_max   367995 non-null  float64       
 10  SNOW_mean  367995 non-null  float64       
 11  SNWD_min   329356 non-null  float64       
 12  SNWD_max   329356 non-null  float64       
 13  SNWD_mean  329356 non-null  float64       
dtypes: datetime64[ns](1), float64(12), int64(1)
memory usage: 44.4 MB


In [40]:
dfw_summary.fillna(method='ffill', inplace=True)
dfw_summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387739 entries, 0 to 387738
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   fips_code  387739 non-null  int64         
 1   date       387739 non-null  datetime64[ns]
 2   PRCP_min   387739 non-null  float64       
 3   PRCP_max   387739 non-null  float64       
 4   PRCP_mean  387739 non-null  float64       
 5   TEMP_min   387739 non-null  float64       
 6   TEMP_max   387739 non-null  float64       
 7   TEMP_mean  387739 non-null  float64       
 8   SNOW_min   387739 non-null  float64       
 9   SNOW_max   387739 non-null  float64       
 10  SNOW_mean  387739 non-null  float64       
 11  SNWD_min   387739 non-null  float64       
 12  SNWD_max   387739 non-null  float64       
 13  SNWD_mean  387739 non-null  float64       
dtypes: datetime64[ns](1), float64(12), int64(1)
memory usage: 44.4 MB


In [41]:
os.getcwd()

'/Users/sk/Library/CloudStorage/GoogleDrive-sachinkumar.srinivasamurthy@sjsu.edu/Shared drives/DATA 228 Big Data/Project/data/AWS/glueInput'

In [48]:
dfw['year']= dfw['date'].dt.year

In [51]:
dfw_summary['year']= dfw_summary['date'].dt.year

In [52]:
dfw_summary.to_csv('../redshift tables/WeatherFACT.csv', index=False)

In [53]:
dfw

,stationID,date,PRCP,SNOW,SNWD,TMAX,TMIN,lat,lng,county_name,fips_code,TEMP,year
0,US1CAAL0001,2008-10-07,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
1,US1CAAL0001,2008-10-08,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
2,US1CAAL0001,2008-10-09,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
3,US1CAAL0001,2008-10-10,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
4,US1CAAL0001,2008-10-11,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6591622,USW00094299,2023-04-22,0.0,NaN,NaN,200.0,-27.0,41.4836,-120.5614,Modoc County,49,86.5,2023
6591623,USW00094299,2023-04-23,0.0,NaN,NaN,194.0,50.0,41.4836,-120.5614,Modoc County,49,122.0,2023
6591624,USW00094299,2023-04-24,0.0,NaN,NaN,167.0,-16.0,41.4836,-120.5614,Modoc County,49,75.5,2023
6591625,USW00094299,2023-04-25,0.0,NaN,NaN,200.0,-38.0,41.4836,-120.5614,Modoc County,49,81.0,2023


In [47]:
dfw.columns

Index(['stationID', 'date', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'lat',
       'lng', 'county_name', 'fips_code', 'TEMP'],
      dtype='object')

In [54]:
stations= dfw[['stationID', 'lat', 'lng', 'fips_code']]
stations.drop_duplicates(inplace=True)
stations.reset_index(inplace=True, drop=True)
stations

/var/folders/q4/8qvknrhs5d9dq4q2zxqgqzxr0000gn/T/ipykernel_9246/764942737.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations.drop_duplicates(inplace=True)


,stationID,lat,lng,fips_code
0,US1CAAL0001,37.8123,-122.2160,1
1,US1CAAL0002,37.7075,-122.0687,1
2,US1CAAL0003,37.7169,-122.0585,1
3,US1CAAL0004,37.6483,-121.8745,1
4,US1CAAL0006,37.8319,-122.2503,1
...,...,...,...,...
2062,USW00093241,38.3775,-121.9586,95
2063,USW00093242,36.9850,-120.1106,39
2064,USW00093243,37.2381,-120.8825,47
2065,USW00093245,38.3208,-123.0747,97


In [112]:
dfw_summary

,fips_code,date,PRCP_min,PRCP_max,PRCP_mean,TEMP_min,TEMP_max,TEMP_mean,SNOW_min,SNOW_max,SNOW_mean,SNWD_min,SNWD_max,SNWD_mean,year
0,1,2005-01-01,23.0,109.0,58.9,22.5,105.5,82.571429,0.0,0.0,0.0,0.0,0.0,0.0,2005
1,1,2005-01-02,43.0,264.0,136.8,17.0,105.5,69.857143,0.0,0.0,0.0,0.0,0.0,0.0,2005
2,1,2005-01-03,0.0,117.0,38.7,22.0,89.0,69.392857,0.0,0.0,0.0,0.0,0.0,0.0,2005
3,1,2005-01-04,0.0,13.0,2.3,19.5,88.5,73.357143,0.0,0.0,0.0,0.0,0.0,0.0,2005
4,1,2005-01-05,0.0,13.0,3.5,-122.5,92.0,55.153846,0.0,0.0,0.0,0.0,0.0,0.0,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387734,115,2023-04-23,0.0,0.0,0.0,156.0,200.0,178.000000,0.0,0.0,0.0,0.0,0.0,0.0,2023
387735,115,2023-04-24,0.0,0.0,0.0,158.5,186.0,172.250000,0.0,0.0,0.0,0.0,0.0,0.0,2023
387736,115,2023-04-25,0.0,0.0,0.0,178.0,197.0,187.500000,0.0,0.0,0.0,0.0,0.0,0.0,2023
387737,115,2023-04-26,0.0,0.0,0.0,197.0,197.5,197.250000,0.0,0.0,0.0,0.0,0.0,0.0,2023


In [110]:
dfw

,stationID,date,PRCP,SNOW,SNWD,TMAX,TMIN,lat,lng,county_name,fips_code,TEMP,year
0,US1CAAL0001,2008-10-07,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
1,US1CAAL0001,2008-10-08,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
2,US1CAAL0001,2008-10-09,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
3,US1CAAL0001,2008-10-10,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
4,US1CAAL0001,2008-10-11,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,Alameda County,1,NaN,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6591622,USW00094299,2023-04-22,0.0,NaN,NaN,200.0,-27.0,41.4836,-120.5614,Modoc County,49,86.5,2023
6591623,USW00094299,2023-04-23,0.0,NaN,NaN,194.0,50.0,41.4836,-120.5614,Modoc County,49,122.0,2023
6591624,USW00094299,2023-04-24,0.0,NaN,NaN,167.0,-16.0,41.4836,-120.5614,Modoc County,49,75.5,2023
6591625,USW00094299,2023-04-25,0.0,NaN,NaN,200.0,-38.0,41.4836,-120.5614,Modoc County,49,81.0,2023


In [111]:
stations.to_csv('../redshift tables/WeatherStationsTABLE.csv', index=False)

# Flood Events

In [56]:
dff= pd.read_csv('caFloodEventsDayLevel.csv')
dff

,event_id,cz_name_str,begin_location,begin_date,event_type,cz_fips,flood_cause,begin_lat,begin_lon,flood_impact_in_days
0,5437987,Riverside County,SUN CITY,2005-01-03,Flash Flood,65,Heavy Rain,33.68333,-117.18333,0
1,5436478,Inyo County,DEATH VLY JCT,2005-01-03,Flash Flood,27,Heavy Rain,36.30000,-116.41667,0
2,5438001,Riverside County,LAKE ELSINORE,2005-01-03,Flash Flood,65,Heavy Rain,33.70000,-117.38333,0
3,5437645,Ventura County,COUNTYWIDE,2005-01-03,Flash Flood,111,Heavy Rain,34.48333,-118.05000,0
4,5437646,Los Angeles County,COUNTYWIDE,2005-01-03,Flash Flood,37,Heavy Rain,34.33333,-118.36667,0
...,...,...,...,...,...,...,...,...,...,...
3812,1065830,Merced County,EL NIDO,2023-01-16,Flood,47,Heavy Rain,37.19000,-120.56000,0
3813,1065825,Kern County,KERNELL,2023-01-16,Flood,29,Heavy Rain,35.77520,-119.30250,0
3814,1069365,San Diego County,VALLEY CENTER,2023-01-16,Flash Flood,73,Heavy Rain,33.31360,-116.98360,0
3815,1065833,Tulare County,DINUBA SEQUOIA ARPT,2023-01-17,Flood,107,Heavy Rain,36.45890,-119.33050,0


In [57]:
len(dff['cz_fips'].unique())

58

In [58]:
dff[dff['cz_name_str'].str.contains('countycounty')]

,event_id,cz_name_str,begin_location,begin_date,event_type,cz_fips,flood_cause,begin_lat,begin_lon,flood_impact_in_days
420,200445,San Franciscountycounty,SAN FRANCISCO,2009-10-13,Flood,75,Heavy Rain,37.7754,-122.4195,0
421,199734,San Franciscountycounty,OCEAN VIEW,2009-10-13,Flood,75,Heavy Rain,37.7369,-122.4046,0
446,213611,San Franciscountycounty,OCEAN VIEW,2010-01-19,Flood,75,Heavy Rain,37.7245,-122.4020,0
815,425211,San Franciscountycounty,SAN FRANCISCO,2012-12-02,Flood,75,Heavy Rain,37.7735,-122.4086,0
820,425210,San Franciscountycounty,OCEAN VIEW,2012-12-02,Flood,75,Heavy Rain,37.7400,-122.4100,0
1003,503749,San Franciscountycounty,SAN FRANCISCO,2014-02-28,Flood,75,Heavy Rain,37.7700,-122.4200,0
1004,503756,San Franciscountycounty,OCEAN VIEW,2014-02-28,Flood,75,Heavy Rain,37.7360,-122.4106,0
1135,551345,San Franciscountycounty,OCEAN VIEW,2014-12-02,Flood,75,Heavy Rain,37.7243,-122.4020,0
1151,551358,San Franciscountycounty,CRISSY AAF,2014-12-03,Flood,75,Heavy Rain,37.8103,-122.3661,0
1153,551350,San Franciscountycounty,SAN FRANCISCO,2014-12-03,Flood,75,Heavy Rain,37.7504,-122.4050,0


In [59]:
dff.loc[dff['cz_name_str'].str.contains('countycounty'), 'cz_name_str'] = 'San Francisco County'

In [60]:
dff[dff['cz_name_str'].str.contains('countycounty')]

,event_id,cz_name_str,begin_location,begin_date,event_type,cz_fips,flood_cause,begin_lat,begin_lon,flood_impact_in_days


In [61]:
dff[dff['cz_name_str'].str.contains('San Francisco')]

,event_id,cz_name_str,begin_location,begin_date,event_type,cz_fips,flood_cause,begin_lat,begin_lon,flood_impact_in_days
420,200445,San Francisco County,SAN FRANCISCO,2009-10-13,Flood,75,Heavy Rain,37.7754,-122.4195,0
421,199734,San Francisco County,OCEAN VIEW,2009-10-13,Flood,75,Heavy Rain,37.7369,-122.4046,0
446,213611,San Francisco County,OCEAN VIEW,2010-01-19,Flood,75,Heavy Rain,37.7245,-122.4020,0
815,425211,San Francisco County,SAN FRANCISCO,2012-12-02,Flood,75,Heavy Rain,37.7735,-122.4086,0
820,425210,San Francisco County,OCEAN VIEW,2012-12-02,Flood,75,Heavy Rain,37.7400,-122.4100,0
1003,503749,San Francisco County,SAN FRANCISCO,2014-02-28,Flood,75,Heavy Rain,37.7700,-122.4200,0
1004,503756,San Francisco County,OCEAN VIEW,2014-02-28,Flood,75,Heavy Rain,37.7360,-122.4106,0
1135,551345,San Francisco County,OCEAN VIEW,2014-12-02,Flood,75,Heavy Rain,37.7243,-122.4020,0
1151,551358,San Francisco County,CRISSY AAF,2014-12-03,Flood,75,Heavy Rain,37.8103,-122.3661,0
1153,551350,San Francisco County,SAN FRANCISCO,2014-12-03,Flood,75,Heavy Rain,37.7504,-122.4050,0


In [62]:
dff

,event_id,cz_name_str,begin_location,begin_date,event_type,cz_fips,flood_cause,begin_lat,begin_lon,flood_impact_in_days
0,5437987,Riverside County,SUN CITY,2005-01-03,Flash Flood,65,Heavy Rain,33.68333,-117.18333,0
1,5436478,Inyo County,DEATH VLY JCT,2005-01-03,Flash Flood,27,Heavy Rain,36.30000,-116.41667,0
2,5438001,Riverside County,LAKE ELSINORE,2005-01-03,Flash Flood,65,Heavy Rain,33.70000,-117.38333,0
3,5437645,Ventura County,COUNTYWIDE,2005-01-03,Flash Flood,111,Heavy Rain,34.48333,-118.05000,0
4,5437646,Los Angeles County,COUNTYWIDE,2005-01-03,Flash Flood,37,Heavy Rain,34.33333,-118.36667,0
...,...,...,...,...,...,...,...,...,...,...
3812,1065830,Merced County,EL NIDO,2023-01-16,Flood,47,Heavy Rain,37.19000,-120.56000,0
3813,1065825,Kern County,KERNELL,2023-01-16,Flood,29,Heavy Rain,35.77520,-119.30250,0
3814,1069365,San Diego County,VALLEY CENTER,2023-01-16,Flash Flood,73,Heavy Rain,33.31360,-116.98360,0
3815,1065833,Tulare County,DINUBA SEQUOIA ARPT,2023-01-17,Flood,107,Heavy Rain,36.45890,-119.33050,0


In [64]:
dff.columns

Index(['event_id', 'cz_name_str', 'begin_location', 'begin_date', 'event_type',
       'cz_fips', 'flood_cause', 'begin_lat', 'begin_lon',
       'flood_impact_in_days'],
      dtype='object')

In [65]:
dff.columns=['event_id', 'county_name', 'location', 'date', 'event_type',
       'fips_code', 'flood_cause', 'lat', 'lng',
       'flood_impact_days']

dff

,event_id,county_name,location,date,event_type,fips_code,flood_cause,lat,lng,flood_impact_days
0,5437987,Riverside County,SUN CITY,2005-01-03,Flash Flood,65,Heavy Rain,33.68333,-117.18333,0
1,5436478,Inyo County,DEATH VLY JCT,2005-01-03,Flash Flood,27,Heavy Rain,36.30000,-116.41667,0
2,5438001,Riverside County,LAKE ELSINORE,2005-01-03,Flash Flood,65,Heavy Rain,33.70000,-117.38333,0
3,5437645,Ventura County,COUNTYWIDE,2005-01-03,Flash Flood,111,Heavy Rain,34.48333,-118.05000,0
4,5437646,Los Angeles County,COUNTYWIDE,2005-01-03,Flash Flood,37,Heavy Rain,34.33333,-118.36667,0
...,...,...,...,...,...,...,...,...,...,...
3812,1065830,Merced County,EL NIDO,2023-01-16,Flood,47,Heavy Rain,37.19000,-120.56000,0
3813,1065825,Kern County,KERNELL,2023-01-16,Flood,29,Heavy Rain,35.77520,-119.30250,0
3814,1069365,San Diego County,VALLEY CENTER,2023-01-16,Flash Flood,73,Heavy Rain,33.31360,-116.98360,0
3815,1065833,Tulare County,DINUBA SEQUOIA ARPT,2023-01-17,Flood,107,Heavy Rain,36.45890,-119.33050,0


In [66]:
fact1= dff[['fips_code', 'date']]
fact1

,fips_code,date
0,65,2005-01-03
1,27,2005-01-03
2,65,2005-01-03
3,111,2005-01-03
4,37,2005-01-03
...,...,...
3812,47,2023-01-16
3813,29,2023-01-16
3814,73,2023-01-16
3815,107,2023-01-17


In [67]:
fact1['floods_count'] = fact1.groupby(['date', 'fips_code'])['fips_code'].transform('count')
fact1

/var/folders/q4/8qvknrhs5d9dq4q2zxqgqzxr0000gn/T/ipykernel_9246/2574962845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact1['floods_count'] = fact1.groupby(['date', 'fips_code'])['fips_code'].transform('count')


,fips_code,date,floods_count
0,65,2005-01-03,2
1,27,2005-01-03,1
2,65,2005-01-03,2
3,111,2005-01-03,1
4,37,2005-01-03,1
...,...,...,...
3812,47,2023-01-16,3
3813,29,2023-01-16,2
3814,73,2023-01-16,9
3815,107,2023-01-17,1


In [68]:
fact1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3817 entries, 0 to 3816
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   fips_code     3817 non-null   int64 
 1   date          3817 non-null   object
 2   floods_count  3817 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 89.6+ KB


In [69]:
fact1.drop_duplicates(inplace=True)
fact1.reset_index(inplace=True, drop=True)
fact1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1981 entries, 0 to 1980
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   fips_code     1981 non-null   int64 
 1   date          1981 non-null   object
 2   floods_count  1981 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 46.6+ KB


/var/folders/q4/8qvknrhs5d9dq4q2zxqgqzxr0000gn/T/ipykernel_9246/1455215890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact1.drop_duplicates(inplace=True)


In [70]:
len(fact1['fips_code'].unique())

58

In [78]:
fact1['floods_count'].value_counts()

1     1196
2      396
3      181
4       71
5       55
6       25
7       15
8       12
9       11
10       4
12       3
15       3
13       2
14       2
11       2
26       1
19       1
35       1
Name: floods_count, dtype: int64

In [ ]:
# 58*18*365
# pivot = fact1.pivot_table(index='fips_code', columns='date', values='floods_count', fill_value=0)

# dates = pd.date_range(start=fact1['date'].min(), end=fact1['date'].max(), freq='D')
# unique_fips = fact1['fips_code'].unique()
# new_df = pd.DataFrame(index=unique_fips, columns=dates)

# # Update the new dataframe with the pivot table values
# new_df.update(pivot)

# # Fill in the NaN values with 0
# new_df = new_df.fillna(0)
# new_df
# new_df.reset_index(inplace=True)
# new_df
# new_df.rename(columns={'index':'fips_code'}, inplace=True)
# import pandas as pd

# # assume the result DataFrame is named 'result_df'
# unpivoted_df = pd.melt(new_df, 
#                        id_vars=['fips_code'], 
#                        var_name='date',
#                        value_name='floods_count')

# # convert date column to datetime format
# unpivoted_df['date'] = pd.to_datetime(unpivoted_df['date'])

# # sort by fips_code and date
# unpivoted_df = unpivoted_df.sort_values(['fips_code', 'date'])

# # reset index
# unpivoted_df = unpivoted_df.reset_index(drop=True)

# # display the result
# unpivoted_df
# unpivoted_df.columns=['fips_code', 'date', 'flood_count']
# unpivoted_df['flood_count'].value_counts()
# fact1
# fact1['date']= fact1['date'].astype('str')
# unpivoted_df['date']= unpivoted_df['date'].astype('str')
# floodFact= pd.merge(unpivoted_df, fact1, how='left')
# floodFact.info()

In [97]:
floodFact['floods_count'].fillna(0, inplace=True)
floodFact['floods_count'].value_counts()

0.0     380935
1.0       1196
2.0        396
3.0        181
4.0         71
5.0         55
6.0         25
7.0         15
8.0         12
9.0         11
10.0         4
12.0         3
15.0         3
11.0         2
13.0         2
14.0         2
35.0         1
19.0         1
26.0         1
Name: floods_count, dtype: int64

In [100]:
fact1.to_csv('../redshift tables/FloodsFACT.csv', index=False)

In [101]:
# floodFact.to_csv('../redshift tables/FloodsFACT_allDates.csv', index=False)

In [102]:
dff

,event_id,county_name,location,date,event_type,fips_code,flood_cause,lat,lng,flood_impact_days
0,5437987,Riverside County,SUN CITY,2005-01-03,Flash Flood,65,Heavy Rain,33.68333,-117.18333,0
1,5436478,Inyo County,DEATH VLY JCT,2005-01-03,Flash Flood,27,Heavy Rain,36.30000,-116.41667,0
2,5438001,Riverside County,LAKE ELSINORE,2005-01-03,Flash Flood,65,Heavy Rain,33.70000,-117.38333,0
3,5437645,Ventura County,COUNTYWIDE,2005-01-03,Flash Flood,111,Heavy Rain,34.48333,-118.05000,0
4,5437646,Los Angeles County,COUNTYWIDE,2005-01-03,Flash Flood,37,Heavy Rain,34.33333,-118.36667,0
...,...,...,...,...,...,...,...,...,...,...
3812,1065830,Merced County,EL NIDO,2023-01-16,Flood,47,Heavy Rain,37.19000,-120.56000,0
3813,1065825,Kern County,KERNELL,2023-01-16,Flood,29,Heavy Rain,35.77520,-119.30250,0
3814,1069365,San Diego County,VALLEY CENTER,2023-01-16,Flash Flood,73,Heavy Rain,33.31360,-116.98360,0
3815,1065833,Tulare County,DINUBA SEQUOIA ARPT,2023-01-17,Flood,107,Heavy Rain,36.45890,-119.33050,0


In [104]:
dff.columns

Index(['event_id', 'county_name', 'location', 'date', 'event_type',
       'fips_code', 'flood_cause', 'lat', 'lng', 'flood_impact_days'],
      dtype='object')

In [106]:
dff= dff[['event_id', 'date', 'fips_code', 'county_name', 'location', 'lat', 'lng',
          'event_type', 'flood_cause', 'flood_impact_days']]
dff

,event_id,date,fips_code,county_name,location,lat,lng,event_type,flood_cause,flood_impact_days
0,5437987,2005-01-03,65,Riverside County,SUN CITY,33.68333,-117.18333,Flash Flood,Heavy Rain,0
1,5436478,2005-01-03,27,Inyo County,DEATH VLY JCT,36.30000,-116.41667,Flash Flood,Heavy Rain,0
2,5438001,2005-01-03,65,Riverside County,LAKE ELSINORE,33.70000,-117.38333,Flash Flood,Heavy Rain,0
3,5437645,2005-01-03,111,Ventura County,COUNTYWIDE,34.48333,-118.05000,Flash Flood,Heavy Rain,0
4,5437646,2005-01-03,37,Los Angeles County,COUNTYWIDE,34.33333,-118.36667,Flash Flood,Heavy Rain,0
...,...,...,...,...,...,...,...,...,...,...
3812,1065830,2023-01-16,47,Merced County,EL NIDO,37.19000,-120.56000,Flood,Heavy Rain,0
3813,1065825,2023-01-16,29,Kern County,KERNELL,35.77520,-119.30250,Flood,Heavy Rain,0
3814,1069365,2023-01-16,73,San Diego County,VALLEY CENTER,33.31360,-116.98360,Flash Flood,Heavy Rain,0
3815,1065833,2023-01-17,107,Tulare County,DINUBA SEQUOIA ARPT,36.45890,-119.33050,Flood,Heavy Rain,0


In [108]:
dff.sort_values(by='date', inplace=True)
dff.reset_index(inplace=True, drop=True)
dff

,event_id,date,fips_code,county_name,location,lat,lng,event_type,flood_cause,flood_impact_days
0,5437987,2005-01-03,65,Riverside County,SUN CITY,33.68333,-117.18333,Flash Flood,Heavy Rain,0
1,5436478,2005-01-03,27,Inyo County,DEATH VLY JCT,36.30000,-116.41667,Flash Flood,Heavy Rain,0
2,5438001,2005-01-03,65,Riverside County,LAKE ELSINORE,33.70000,-117.38333,Flash Flood,Heavy Rain,0
3,5437645,2005-01-03,111,Ventura County,COUNTYWIDE,34.48333,-118.05000,Flash Flood,Heavy Rain,0
4,5437646,2005-01-03,37,Los Angeles County,COUNTYWIDE,34.33333,-118.36667,Flash Flood,Heavy Rain,0
...,...,...,...,...,...,...,...,...,...,...
3812,1065824,2023-01-16,47,Merced County,GUSTINE,37.24620,-121.01210,Flood,Heavy Rain,0
3813,1065822,2023-01-16,19,Fresno County,ALDER SPGS,37.06550,-119.40230,Flood,Heavy Rain,0
3814,1069369,2023-01-16,73,San Diego County,ENCINITAS,33.03340,-117.25220,Flood,Heavy Rain,0
3815,1065833,2023-01-17,107,Tulare County,DINUBA SEQUOIA ARPT,36.45890,-119.33050,Flood,Heavy Rain,0


In [109]:
dff.to_csv('../redshift tables/floodEvents_TABLE.csv', index=False)

# Insurance

In [113]:
df= pd.read_csv('caFimaNfipInsuranceRecordLevel2009to2022.csv')
df

,id,policyEffectiveDate,policyYear,policyCost,policyCount,countyCode,fips_code,county_name,reportedZipCode,floodZone,latitude,longitude,occupancyType,floorsCount,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,totalInsurancePremiumOfThePolicy
0,4e38a265-92ed-40ad-9fe2-b909180f3a86,2009-01-01,2009,296,1,6101,1,Alameda,95957,X,39.1,-121.9,1.0,1.0,150000.0,60000.0,283.0
1,5810ffbe-5d9f-4e72-8d9f-583f886bd153,2009-01-01,2009,348,1,6101,1,Alameda,95991,X,39.1,-121.6,1.0,1.0,250000.0,100000.0,335.0
2,1c38a91f-2bbf-4fe8-ac32-6d1f1fefe519,2009-01-01,2009,348,1,6101,1,Alameda,95993,X,39.1,-121.6,1.0,1.0,250000.0,100000.0,335.0
3,38bd25a3-2e61-473b-b198-33239988bb3a,2009-01-01,2009,348,1,6101,1,Alameda,95668,X,38.9,-121.5,1.0,1.0,250000.0,100000.0,335.0
4,3b6b6139-5e2a-42f8-8013-f0d4ee3f892e,2009-01-01,2009,348,1,6101,1,Alameda,95659,X,38.9,-121.6,1.0,1.0,250000.0,100000.0,335.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3311348,e648231a-87ab-40b2-ab63-43b366677e39,2022-05-03,2022,1836,1,6099,99,Stanislaus,95356,X,37.7,-121.0,18.0,1.0,500000.0,158000.0,1304.0
3311349,48f427fa-f9c5-4e35-a5ef-4c03030dee9c,2022-05-09,2022,998,1,6099,99,Stanislaus,95360,AO,37.3,-121.0,18.0,1.0,277000.0,75000.0,594.0
3311350,14c8bb3a-e3af-4f60-8f13-9c32852005fc,2022-05-11,2022,688,1,6099,99,Stanislaus,95360,X,37.3,-121.0,11.0,1.0,250000.0,100000.0,522.0
3311351,999e2d4b-8a18-47da-8223-306f13bb4434,2022-05-11,2022,702,1,6099,99,Stanislaus,95360,AO,37.3,-121.0,1.0,1.0,207000.0,0.0,534.0


In [114]:
len(df['fips_code'].unique())

49

In [115]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3311353 entries, 0 to 3311352
Data columns (total 17 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   id                                3311353 non-null  object 
 1   policyEffectiveDate               3311353 non-null  object 
 2   policyYear                        3311353 non-null  int64  
 3   policyCost                        3311353 non-null  int64  
 4   policyCount                       3311353 non-null  int64  
 5   countyCode                        3311353 non-null  int64  
 6   fips_code                         3311353 non-null  int64  
 7   county_name                       3311353 non-null  object 
 8   reportedZipCode                   3311353 non-null  int64  
 9   floodZone                         3311353 non-null  object 
 10  latitude                          3311353 non-null  float64
 11  longitude                         331

In [116]:
df['insuranceCoverage']= df['totalBuildingInsuranceCoverage']+df['totalContentsInsuranceCoverage']

## Table insurance

In [117]:
a= df.groupby(['fips_code', 'policyYear'])['insuranceCoverage'].apply(lambda x: np.log(x.sum())).reset_index(name='log_insured')
b= df.groupby(['fips_code', 'policyYear'])['policyCost'].apply(lambda x: np.log(x.sum())).reset_index(name='log_cost')
c= df.groupby(['policyYear', 'county_name']).size().reset_index(name='policy_count')

In [118]:
dfi= a.merge(b).merge(c)
dfi

,fips_code,policyYear,log_insured,log_cost,county_name,policy_count
0,1,2009,21.896683,15.756230,Alameda,11270
1,1,2009,21.896683,15.756230,Alpine,1248
2,1,2009,21.896683,15.756230,Amador,366
3,1,2009,21.896683,15.756230,Butte,14202
4,1,2009,21.896683,15.756230,Calaveras,219
...,...,...,...,...,...,...
33609,99,2022,18.004853,12.648397,Sierra,15
33610,99,2022,18.004853,12.648397,Siskiyou,87
33611,99,2022,18.004853,12.648397,Solano,630
33612,99,2022,18.004853,12.648397,Sonoma,826


In [119]:
dfi.rename(columns={'policyYear':'year'}, inplace=True)

In [121]:
os.getcwd()

'/Users/sk/Library/CloudStorage/GoogleDrive-sachinkumar.srinivasamurthy@sjsu.edu/Shared drives/DATA 228 Big Data/Project/data/AWS/glueInput'

In [122]:
dfi.to_csv('../redshift tables/InsuranceTABLE.csv', index=False)

## Table FloodZone

In [123]:
df.head()

,id,policyEffectiveDate,policyYear,policyCost,policyCount,countyCode,fips_code,county_name,reportedZipCode,floodZone,latitude,longitude,occupancyType,floorsCount,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,totalInsurancePremiumOfThePolicy,insuranceCoverage
0,4e38a265-92ed-40ad-9fe2-b909180f3a86,2009-01-01,2009,296,1,6101,1,Alameda,95957,X,39.1,-121.9,1.0,1.0,150000.0,60000.0,283.0,210000.0
1,5810ffbe-5d9f-4e72-8d9f-583f886bd153,2009-01-01,2009,348,1,6101,1,Alameda,95991,X,39.1,-121.6,1.0,1.0,250000.0,100000.0,335.0,350000.0
2,1c38a91f-2bbf-4fe8-ac32-6d1f1fefe519,2009-01-01,2009,348,1,6101,1,Alameda,95993,X,39.1,-121.6,1.0,1.0,250000.0,100000.0,335.0,350000.0
3,38bd25a3-2e61-473b-b198-33239988bb3a,2009-01-01,2009,348,1,6101,1,Alameda,95668,X,38.9,-121.5,1.0,1.0,250000.0,100000.0,335.0,350000.0
4,3b6b6139-5e2a-42f8-8013-f0d4ee3f892e,2009-01-01,2009,348,1,6101,1,Alameda,95659,X,38.9,-121.6,1.0,1.0,250000.0,100000.0,335.0,350000.0


In [124]:
a= pd.DataFrame(df[['fips_code', 'policyYear', 'floodZone']].value_counts())
a.columns=['policyCount']
a.reset_index(inplace=True)
a.sort_values(by=['fips_code', 'policyYear', 'policyCount'], ascending=[True,True, False], inplace=True)
a.reset_index(inplace=True, drop=True)
a

,fips_code,policyYear,floodZone,policyCount
0,1,2009,X,6394
1,1,2009,C,2160
2,1,2009,AE,1185
3,1,2009,AO,419
4,1,2009,A,274
...,...,...,...,...
9883,99,2022,B,14
9884,99,2022,A,8
9885,99,2022,A24,3
9886,99,2022,C,2


In [125]:
a.columns=['fips_code', 'year', 'floodZone', 'policyCount']
a

,fips_code,year,floodZone,policyCount
0,1,2009,X,6394
1,1,2009,C,2160
2,1,2009,AE,1185
3,1,2009,AO,419
4,1,2009,A,274
...,...,...,...,...
9883,99,2022,B,14
9884,99,2022,A,8
9885,99,2022,A24,3
9886,99,2022,C,2


In [126]:
a.to_csv('../redshift tables/FloodZoneTABLE.csv', index=False)